<a href="https://colab.research.google.com/github/fatfat11111111/bt/blob/main/AMR_B_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
a = []
while(1):
  a.append('1')

# Post processing of mumax3 output data in python

 <a id='gettingready'></a>
## 1. Getting ready

If you want to run the examples in this notebook, then mumax3 must be is installed on the system that is running this notebook. Go to [mumax.github.io/download](http://mumax.github.io/download) for more information on how to **install mumax3** on your system. If you are running this notebook in a google colaboratory session, then it suffices to run the cell below to install mumax3. 

In [ ]:
try: # 使用 try，測試內容是否正確
    import google.colab
except ImportError:  # 如果 try 的內容發生錯誤，就執行 except 裡的內容
    pass
else: #完全沒有錯誤，就會執行該區塊的程式
    !wget https://mumax.ugent.be/mumax3-binaries/mumax3.10_linux_cuda10.1.tar.gz #下載網路上的檔案，可執行 wget 加上檔案的網址即可立即下載
    !tar -xvf mumax3.10_linux_cuda10.1.tar.gz #tar -xvf解包
    !rm mumax3.10_linux_cuda10.1.tar.gz
    !rm -rf mumax3.10 && mv mumax3.10_linux_cuda10.1 mumax3.10 #刪除檔案或目錄
    import os #OS模塊，調用操作系統命令，來達成建立文件，刪除文件，查詢文件等
    os.environ['PATH'] += ":/content/mumax3.10" #執行時將程式的所有環境變數保留在類似字典的 os.environ 物件
    from google.colab import drive #掛載雲端
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/mumax2.0') #切換該目錄
    os.listdir() #確認目錄內容


In the examples presented in this notebook, we will use the numpy and pandas libraries **for post processing mumax3 output data**, and matplotlib to **visualize this data**. So let's import these libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

 <a id='helperfunctions'></a>
## 2. mumax3 helper functions

Let's start by writing a function which **reads a mumax3 output table** and puts the date in a **Pandas dataframe**.

In [ ]:
def read_mumax3_table(filename): # 定義函式
    """Puts the mumax3 output table in a pandas dataframe""" # pandas dataframe可以處理雙維度或多欄位的資料

    from pandas import read_table # read_table載入帶分隔符的資料
    
    table = read_table(filename)
    table.columns = ' '.join(table.columns).split()[1::2]
    
    return table

Mumax3 does not only write output data in a table, it can also write field data to ovf files. The function below **converts all ovf files in the output directory to numpy files**. These files are then loaded using the numpy.load function. The result is a **python dictionary of the field data (a numpy array) with the ovf filename as key**.

In [ ]:
def read_mumax3_ovffiles(outputdir):
    """Load all ovffiles in outputdir into a dictionary of numpy arrays 
    with the ovffilename (without extension) as key"""
    
    from subprocess import run, PIPE, STDOUT # subprocess可以在當前程序中執行其他程序或命令 https://medium.com/@alan81920/python-%E5%91%BC%E5%8F%AB%E5%85%B6%E4%BB%96%E7%A8%8B%E5%BC%8F-18a202c8b700
    from glob import glob # 查找出匹配條件的檔案或資料夾
    from os import path # os.path() 獲取文件屬性
    from numpy import load # load 讀取.npy檔數據的函數，默認情况下，是以未壓缩的原始二進制格式保存

    # convert all ovf files in the output directory to numpy files, npy檔用於儲存重建ndarray 所需的數據、圖形、dtype 和其他資訊
    p = run(["mumax3-convert","-numpy",outputdir+"/*.ovf"], stdout=PIPE, stderr=STDOUT)
    if p.returncode != 0:
        print(p.stdout.decode('UTF-8'))

    # read the numpy files (the converted ovf files)
    fields = {}
    for npyfile in glob(outputdir+"/*.npy"):
        key = path.splitext(path.basename(npyfile))[0] #os.path.splitext(“檔案路徑”)分離檔名與副檔名
        fields[key] = load(npyfile)
    
    return fields

The function below **executes a mumax3 script** and r**eturns the data of the output table in a pandas dataframe**, and the **saved fields as numpy arrays**.
*Usage of mumax3-convert:
-arrows int
Arrow size for vector bitmap image output
-color string
Colormap for scalar image output. (default "black,gray,white")
-comp string
Select a component of vector data. (0,1,2 or x,y,z)
-csv
output in CSV format
-dump
output in dump format
-f string
Printf format string (default "%v")
-gif
GIF output
-gplot
Gnuplot-compatible output
-jpg
JPEG output
-json
output in JSON format
-max string
Maximum of color scale: "auto" or value. (default "auto")
-min string
Minimum of color scale: "auto" or value. (default "auto")
-normalize
Normalize vector data to unit length
-normpeak
Scale vector data, maximum to unit length
-numpy
Numpy output
-o string
Save all output in this directory
-omf string
"text" or "binary" OVF1 output
-ovf string
"text" or "binary" OVF1 output
-ovf2 string
"text" or "binary" OVF2 output
-png
PNG output
-resize string
Resize. E.g.: 128x128x4
-show
Human-readible output to stdout
-svg
SVG output
-svgz
SVGZ output (compressed)
-vtk string
"ascii" or "binary" VTK output
-xrange string
Crop x range min:max (both optional, max=exclusive)
-yrange string
Crop y range min:max (both optional, max=exclusive)
-zrange string
Crop z range min:max (both optional, max=exclusive)*

In [ ]:
def run_mumax3(script, name, verbose=False):
    """ Executes a mumax3 script and convert ovf files to numpy files
    
    Parameters
    ----------
      script:  string containing the mumax3 input script
      name:    name of the simulation (this will be the name of the script and output dir)
      verbose: print stdout of mumax3 when it is finished
    """
    
    from subprocess import run, PIPE, STDOUT
    from os import path

    scriptfile = name + ".txt" 
    outputdir  = name + ".out"

    # write the input script in scriptfile
    with open(scriptfile, 'w' ) as f: # with open讀寫檔案 'w'打開一個檔案，只用於寫入，如果檔案已存在則將其覆蓋；反之創建新檔案
        f.write(script)
    
    # call mumax3 to execute this script
    p = run(["mumax3","-f",scriptfile], stdout=PIPE, stderr=STDOUT)
    if verbose or p.returncode != 0:
        print(p.stdout.decode('UTF-8'))
        
    if path.exists(outputdir + "/table.txt"): # 查看特定的路徑是否存在，不分檔案或目錄
        table = read_mumax3_table(outputdir + "/table.txt")
    else:
        table = None
        
    fields = read_mumax3_ovffiles(outputdir)
    
    return table, fields

Let's start by **putting the mumax3 input scrip**t in the string variable `script`.
(二維，一個磁場下)

In [ ]:
script="""
sizeX := 256e-8 // length = 2560 nm
sizeY := 256e-8  // width = 25602 nm
sizeZ := 30e-9 // thickness  = 30 nm
Nx := 256
Ny := 256
Nz := 1
setgridsize(Nx, Ny, Nz)
setcellsize(sizeX/Nx, sizeY/Ny, sizeZ/Nz)
setGeom(rect(sizeX, sizeY))

//defregion(1,rect(sizeX, sizeY))
//Kc1.SetRegion(0, -4.5e+100 ) 
//AnisC1.SetRegion(0, vector(0,1,0) )
//AnisC1.SetRegion(1, vector(0, 1, 0)) 
//Kc1.SetRegion(1, Kkc1)
Kc1 = 500 // 1st order cubic anisotropy constant (J/m3)
AnisC1 = vector(0,1,0) // Cubic anisotropy direction #1
//AnisU = vector(0, 1, 0)
//Ku1 = 500 
//
Msat = 47e+4 // Saturation magnetization (A/m)
Aex = 92e-13 // Exchange stiffness (J/m)
alpha = 0.02
m = randomMag() 

TableAdd(B_ext)
B := 6.5e-2  // 650 Oe
B_ext = vector(B, 0, 0)

autosave(m, 100e-12)
tableautosave(10e-12)




run(10.0e-12)
"""

Now we can **execute this mumax3** script using the `run_mumax3` helper function. This function returns **the output table in a pandas dataframe and a dictionary of the saved fields**. If `verbose=True`, then the log output of mumax3 will be printed out when mumax3 finishes. Note that on the left side of the cell, you can check if the simulation is finished or still running.

In [ ]:
table, fields = run_mumax3( script, name="project3.2", verbose=False )

The table data is put in a pandas dataframe. This makes it very easy to **plot and analyze the table data**.

post processing using python

In [ ]:
from numpy.core.fromnumeric import shape # shape 讀取矩陣的長度
for i in fields.keys():
  m= fields[i]
  
  mx = m[0,:,:,:]
  my = m[1,:,:,:] 
  mz = m[2,:,:,:]
  Nx = int(input("Nx="))
  Ny = int(input("Ny="))
  # m_component= np.array([[mx], [my], [mz]])
  cos_1 = mx #cos= m·j/|m||j|, let j= ( 0, 1, 0)
  cos_1 = cos_1.flatten() #Return a copy of the array collapsed into one dimension.
  cos_1 = np.reshape(cos_1, (Nx, Ny)) #Reshape array into two dimension 
  cos_2 = my
  cos_2 = cos_2.flatten()
  cos_2 = np.reshape(cos_2, (Nx, Ny))
  # print("cos_1=",cos_1)
  cossquare1 = np.power(cos_1, 2)
  cossquare2 = np.power(cos_2, 2)
  # print(cossquare1)
  AMRratio = 0.025 #AMRratio= (lo_xx-lo_xy)/lo_xy
  lo_xx = 6.99*(10**(-8))
  lo_xy = lo_xx/(AMRratio+1)

  Rx= lo_xy*(1+AMRratio*cossquare1)
  Ry= lo_xy*(1+AMRratio*cossquare2) #coductivity of each cell in a field
  # print("Rx=",Rx)
  # print("Ry=",Ry)
  R = Ry-(np.abs(Ry**2-Ry*Rx)**(1/2))
  r = Ry+(np.abs(Ry**2-Ry*Rx)**(1/2))
  # print("R=",R)
  # print("r=",r)


  
  a = 2*Nx*Ny+Nx+Ny
  k = 0

  matrix = np.array([[0 for j in range(a)]for k in range(a)], dtype = 'float')
  for j in range(0, Nx): # 頭幾個
    matrix[j][j+Nx] = r[j][0]
    matrix[j][j+Nx+1] = r[j][0] 
  for j in range(a-Nx, a): # 尾幾個
    for q in range(Nx):
      matrix[j][j-Nx] = R[q][Ny-1]
      matrix[j][j-(Nx+1)] = R[q][Ny-1] 
  for j in range(Nx, a-2*Nx, 2*Nx+1): # 左側
    for p in range(Ny):
      matrix[j][j-Nx] = r[0][p]
      matrix[j][j+(Nx+1)] = R[0][p]  
  for j in range(2*Nx, a-Nx, 2*Nx+1): # 右側
    for p in range(Ny):
      matrix[j][j-(Nx+1)] = r[Nx-1][p]
      matrix[j][j+Nx] = r[Nx-1][p]
  for j in range(a):
    for q in range(Nx):
      for p in range(Ny):
        if ((j not in range(0, Nx)) and (j not in range(a-Nx, a)) and (j not in range(Nx, a-2*Nx, 2*Nx+1)) and (j not in range(2*Nx, a-Nx, 2*Nx+1))):
          matrix[j][j+Nx] = R[q][p]
          matrix[j][j+(Nx+1)] = R[q][p]
          matrix[j][j-Nx] = r[q][p]
          matrix[j][j-(Nx+1)] = r[q][p]
print(repr(matrix))